# 0 - Instalaciones e importaciones previas al ejc


In [ ]:
# En caso de problemas, utilizar las dependencias de librerías de este requierement: https://github.com/googlecolab/backend-info/blob/d6d345cb94fc5fd49951c9af0f6ead5e962bfab2/pip-freeze.txt
!pip install numpy==1.23.5
!pip install transformers[torch]==4.35.2
!pip install accelerate -U
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 1.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; python_version >= "3.9", but you have numpy 1.23.5 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 39.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux

In [ ]:
import pandas as pd
def load_prepare_data(path):
  """
  Función para cargar y procesar datos para el ejercicio.
  """
  df = pd.read_csv(path,sep=",")
  map_classes = {
    "religion":1,
    "age":1,
    "ethnicity":1,
    "gender":1,
    "other_cyberbullying":1,
    "not_cyberbullying":0,
  }
  df["cyberbullying"] = df.cyberbullying_type.map(map_classes)
  return df[["tweet_text","cyberbullying"]].copy()

# Ejercicio


## Descripción inicial

## 0. Imports


In [ ]:
from transformers import (
   AutoConfig,
   AutoTokenizer,
   AutoModelForSequenceClassification,
   AdamW,
   TrainingArguments,
   Trainer
)
import torch
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import accelerate
import numpy as np
import evaluate

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


## 1. Obtención del corpus

In [ ]:
path_data = "https://raw.githubusercontent.com/luisgasco/ntic_master_datos/main/datasets/cyberbullying_tweets.csv"
# Path de datos alternativos en caso de que el anterior no funcione (al estar alojado en github puede haber limitaciones
# en la descarga.
# path_data = "https://zenodo.org/records/10938455/files/cyberbullying_tweets.csv?download=1"
dataset = load_prepare_data(path_data)

Comprobamos que se haya importado correctamente:

In [ ]:
dataset.head(4)

,tweet_text,cyberbullying
0,"In other words #katandandre, your food was cra...",0
1,Why is #aussietv so white? #MKR #theblock #ImA...,0
2,@XochitlSuckkks a classy whore? Or more red ve...,0
3,"@Jason_Gio meh. :P thanks for the heads up, b...",0


## 2. Análisis exploratorio (incluido en Text_Mining1.ipynb)



## 3. Preprocesado y Normalización

Fijamos el nombre del modelo elegido de HuggingFace y generamos el tokenizador.

In [ ]:
model_name ="bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Separamos el dataset en la variable dependiente y la independiente.

In [ ]:
texts = dataset.tweet_text.values  # La columna de texto de los tweets
labels = dataset.cyberbullying.values  # La columna de categorización de bullying

## 4. Vectorización

La necesidad de transformar el dataset a introducir en el modelo (para tokenizar y vectorizar) a una clase adaptada a la libreria *transform*,  nos obliga a separar las muestras con el fin de entrenar el modelo antes incluso de realizar la vectorización.

Esto se debe a la incompatibilidad del modulo train_test_split de la libreria sklearn para actuar sobre la clase generada

Realizamos el train_test_split de la libreria sklearn generando tambien una submuestra de evaluación previa a la predicción

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(texts,
                                                                      labels,
                                                                      test_size=.25,
                                                                      random_state=0,
                                                                      stratify = labels)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts,
                                                                    train_labels,
                                                                    test_size=.2,
                                                                    random_state=0,
                                                                    stratify = train_labels)

Copiamos la clase utilizada para trabajar con el Trainer de la libreria *transform*

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        """
        Constructor de la clase CustomDataset.
        Parámetros:
        - texts: Lista de textos.
        - labels: Lista de etiquetas correspondientes a los textos.
        - tokenizer: Objeto del tokenizador a utilizar.
        - max_length: Longitud máxima de la secuencia después de la tokenización.
        """
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Obtener el texto y la etiqueta del índice proporcionado
        text = str(self.texts[idx])
        label = int(self.labels[idx])

        # Tokenizar el texto
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )

        # Devolver el diccionario con los datos
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

Generamos una clase para cada submuestra generada a usar y fijamos el parametro max_length común para todas las clases.

In [ ]:
max_length = 100
train_dataset = CustomDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer, max_length)
test_dataset = CustomDataset(test_texts, test_labels, tokenizer, max_length)

Comprobamos el formato resultante

In [ ]:
train_dataset[20]

{'input_ids': tensor([  101,  2027,  1005,  2128,  2725,  1996,  4438,  2082, 20716,  5248,
          1012,  2725,  2035,  2027,  2064,  2000,  2131,  2111,  1999,  4390,
          1998,  2131,  2185,  2007,  2009,  2011,  7302,  3093,  2012,  1996,
          7036,  1998,  2667,  2000,  2377,  1996,  7499,  2208,  1012,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 

## 5. Entrenamiento y evaluación de modelos


### Creación del modelo y establecimiento de parametros y variables antes de su ejecución

Generamos el modelo de clasificación fijando etiquetas para interpretar con mayor facilidad el id proveniente de la clase previa.

In [ ]:
id2label = {0: "NO_BULLYING", 1: "BULLYING"}
label2id = {"NO_BULLYING": 0, "BULLYING": 1}
model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           num_labels=2,
                                                           id2label=id2label,
                                                           label2id=label2id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fijamos un diccionario de argumentos para el entrenamiento del modelo.

In [ ]:
training_args = TrainingArguments(
    output_dir="modelo_test",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False
)

Generamos una función basada en elementos de la libreria evaluate para facilitar la evaluación de los resultados del modelo.

In [ ]:
accuracy = evaluate.load("accuracy")
f1_score = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy_value = accuracy.compute(predictions=predictions, references=labels)
    f1_score_value = f1_score.compute(predictions=predictions, references=labels)

    return {
        "accuracy": accuracy_value,
        "f1_score": f1_score_value,
    }

Creamos el modelo bajo el nombre trainer con parametros previamente establecidos

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


### Ejecución del modelo (entrenamiento, evaluación, predicción y análisis de resultados

Entrenamos el modelo con el train_dataset previamente establecido. Observamos metricas como el accuracy con buenos resultados que esperamos se mantengan en la evaluación y el testeo de no haber un sobreajuste.

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.254900,0.252267,{'accuracy': 0.8934861615879228},{'f1': 0.9378973105134474}


Trainer is attempting to log a value of "{'accuracy': 0.8934861615879228}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9378973105134474}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=3577, training_loss=0.28381188381708033, metrics={'train_runtime': 661.5593, 'train_samples_per_second': 43.254, 'train_steps_per_second': 5.407, 'total_flos': 1470492743970000.0, 'train_loss': 0.28381188381708033, 'epoch': 1.0})

Evaluamos el modelo con el val_dataset previamente establecido. Se mantiene un resultado semejante en las métricas.

In [ ]:
# Make predictions on the test data
trainer.evaluate(test_dataset) # evaluate con test_dataset

Trainer is attempting to log a value of "{'accuracy': 0.891050910005871}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9364325911426473}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.26040780544281006,
 'eval_accuracy': {'accuracy': 0.891050910005871},
 'eval_f1_score': {'f1': 0.9364325911426473},
 'eval_runtime': 75.7886,
 'eval_samples_per_second': 157.319,
 'eval_steps_per_second': 19.673,
 'epoch': 1.0}

Predecimos la variable dependiente con el modelo y guardamos sus resultados bajo el nombre de predictions

In [ ]:
predictions = trainer.predict(test_dataset)

Guardamos en una lista los valores de la variable independiente en cada caso predichos por el modelo

In [ ]:
y_pred = predictions.predictions.argmax(axis=1)

Iteramos sobre el valor real de la variable en cada elemento de la muestra con el fin de comparar ambas luego

In [ ]:
y_true = [x["labels"].item() for x in test_dataset]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))

[[1056  930]
 [ 369 9568]]
              precision    recall  f1-score   support

           0       0.74      0.53      0.62      1986
           1       0.91      0.96      0.94      9937

    accuracy                           0.89     11923
   macro avg       0.83      0.75      0.78     11923
weighted avg       0.88      0.89      0.88     11923



Observamos mejores resultados que el modelo con el que concluimos el ejercicio anterior de acuerdo a las métricas de accuracy, precision y recall.

## Conclusión

Observamos mejores resultados que el modelo con el que concluimos el ejercicio anterior de acuerdo a las métricas de accuracy, precision y recall. A falta de saber que tipo de modelo a usado para llevar a cabo la predicción por dificultades para entender la documentación de la libreria, apostaría más por la inclusión de un proceso de stemming de los tokens que por la elección del modelo (si acaso alternativo a los propuestos previamente) como causante de la mejora en la capacidad predictiva del modelo.

Cabe decir, que algunos de los cambios que han sido señalados en el ejercicio anterior como razones que podrían mejorar el modelo (la omisión de simbolos de puntuación, por ejemplo) están muy probablemente incluidos dentro del proceso ejecutado en este ejercicio lo que tambien justifica la mejora.

Aunque aumenta la capacidad predictiva, la opacidad de la información que tenemos de este modelo (seguramente por la falta de dominio de la libreria más que por la falta de herramientas contenidas en esta), como el peso de las características del modelo predictivo, nos deja con una aplicabilidad menor. Nos obliga a ejecutar el modelo en casos como el prefiltrado de denuncias por bullying en la plataforma cuando existiese la posibilidad o fuese necesario por las restricciones técnicas de solo establecer reglas basadas en la presencia de esas características.